<a href="https://colab.research.google.com/github/KIHOON71/pytorch_tutorial/blob/main/pytorch_collate_fn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collatae function

- Dataloader는 기본적으로 자동적으로 batch_size를 파라미터로 받아서 collate를 실행한다.
- 기본적으로 default collate 함수는 기본적으로 데이터들이 어떤 데이터의 타입으로 반환하는가를 확인하고, batch 를 (x_batch, y_batch)로 묶으려고 한다.
- 그러나 데이터 타입에 따라 기본적인 collate함수를 사용할 수 없으면, 우리는 커스텀하여 사용할 수 있다.

### 예시

In [4]:
import random
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torch
from torch.nn.utils.rnn import pad_sequence

In [5]:
reviews = ['No man is an island','Entire of itself',
'Every man is a piece of the continent','part of the main',
'If a clod be washed away by the sea','Europe is the less',
'As well as if a promontory were','As well as if a manor of thy friend',
'Or of thine own were','Any man’s death diminishes me',
'Because I am involved in mankind',
'And therefore never send to know for whom the bell tolls',
'It tolls for thee']

labels = [random.randint(0,1) for i in range(len(reviews))]

dataset = list(zip(reviews, labels))

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
  for text, label in data_iter:
    yield tokenizer(text)


vocab = build_vocab_from_iterator(yield_tokens(iter(dataset)), specials = ['<unk>'])
vocab.set_default_index(vocab['<unk>'])

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

text_pipeline = lambda x: vocab(tokenizer(x))

- 위와 같은 경우에 custom collate 함수가 사용될수 있다. 시퀀셜 데이터 중 가장 긴 길이에 맞춰 padding을 집어 넣기 위해라던지. 
- collate 함수는 매 번 데이터 샘플의 리스트와 함께 호출된다. 
- collate 함수는 input data들을 하나의 batch로 생성하는 역할을 한다.


### custom collate function

In [12]:
def collate_batch(batch):

  label_list, text_list = [], []

  for (_text, _label) in batch:
    label_list.append(_label)
    processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
    text_list.append(processed_text)

  label_list = torch.tensor(label_list, dtype=torch.int64)
  
  text_list = pad_sequence(text_list, batch_first=True, padding_value=0)

  return text_list.to(device), label_list.to(device)

- 다양한 사이즈의 시퀀셜한 데이터를 collate할 때 torch.nn.utils.rnn.pad_sequence를 사용하여 padding을 줄 수 있다.

- collate_fn의 input은 dataloader에 있는 배치 사이즈의 배치 데이터이다.



In [13]:
dataloader = DataLoader(dataset, batch_size=2, collate_fn=collate_batch,shuffle=True)

for x,y in dataloader:
  print(x, "Targets", y, '\n')

tensor([[ 4, 10,  4,  5,  3, 48, 11],
        [25,  1, 34,  0,  0,  0,  0]]) Targets tensor([0, 1]) 

tensor([[ 4, 10,  4,  5,  3, 39,  1, 54, 28,  0,  0],
        [14, 52, 42, 50, 55, 35,  7, 57,  2, 19,  9]]) Targets tensor([1, 0]) 

tensor([[18, 29, 12, 31, 30, 38,  0,  0,  0],
        [ 5,  3, 21, 17, 56, 16, 20,  2, 49]]) Targets tensor([1, 1]) 

tensor([[15, 40, 23, 24, 41],
        [43,  8,  6, 13, 32]]) Targets tensor([0, 0]) 

tensor([[33,  9,  7, 51,  0],
        [44,  1, 53, 45, 11]]) Targets tensor([0, 0]) 

tensor([[26,  6,  2, 36],
        [46,  1,  2, 37]]) Targets tensor([0, 1]) 

tensor([[27,  8,  6,  3, 47,  1,  2, 22]]) Targets tensor([1]) 

